# Natural Language Generation

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
%cd 'drive/MyDrive/NLP_lab_2'

/content/drive/MyDrive/NLP_lab_2


In [8]:
!pip install transformers

In [53]:
from transformers import DataCollatorForLanguageModeling, GPT2LMHeadModel, GPT2Tokenizer, TextDataset, TrainingArguments, Trainer, pipeline

# Датасет
Датасет был составлен на основе текстов авторов немецкой классической философии (Кант, Гегель, Ницше, Фихте). За основу брались главные произведения философов и производилась первичная обработка текстов - очищение от примечаний, сносок, уточнений перевода, удаление оглавлений, нумераций и т.д. Итоговый датасет содержит чистый текст авторов разделенный на небольшие абзацы.

In [114]:
train_path = 'train.txt'
test_path = 'test.txt'

In [139]:
import re

def summary(file_name):
  with open(file_name) as f:
    texts = re.split('[\.!\?]+', f.read())
    words = []
    for txt in texts:
      words += txt.split()

    while True:
      try:
        texts.remove('\n')
      except ValueError:
        break
    
    print(f'Один из примеров:\n {texts[0]}\n')
    print(f'Количество предложений: {len(texts)}')
    print(f'Количество слов: {len(words)}')
    print(f'Количество разных слов: {len(set(words))}')
    print(f'Средняя длина предложения: {sum( map(len, [s.split() for s in texts]) ) / len(texts):.2f} слов')
    print(f'Самый короткий пример: {len(min(texts, key=lambda x: len(x.split())).split())} слов(о)')
    print(min(texts, key=lambda x: len(x)))
    print()
    print(f'Самый длинный пример:  {len(max(texts, key=lambda x: len(x.split())).split())} слов(о)')
    print(max(texts, key=lambda x: len(x.split())))

### Характеристика датасета:

In [140]:
summary(train_path)

Один из примеров:
 Естественно предполагать, что в философии, прежде чем приступить к самой сути дела, то есть к действительному познаванию того, что поистине есть, необходимо заранее договориться относительно познавания, рассматриваемого как орудие, с помощью которого овладевают абсолютным, или как средство, при помощи которого его видят насквозь

Количество предложений: 15412
Количество слов: 421764
Количество разных слов: 44228
Средняя длина предложения: 27.37 слов
Самый короткий пример: 1 слов(о)

Я

Самый длинный пример:  165 слов(о)
 Это с очевидностью доказывается тем, что мы можем представить себе время, которое вовсе не есть предмет внешнего созерцания, не иначе, как имея образ линии, поскольку мы ее проводим, так как без этого способа изображения  мы не могли бы познать единицу его измерения; это можно доказать также ссылкой на то, что определение продолжительности времени, а также место во времени для всех внутренних восприятии всегда должны заимствоваться нами от того измен

# Обработка данных

Будем использовать предобученную модель от Сбера: rugpt3small_based_on_gpt2

In [54]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [117]:
print(f'Размер словаря {tokenizer.vocab_size}')
print(f'Пример токенизации: "немецкая классическая философия": {tokenizer("немецкая классическая философия")}')

Размер словаря 50257
Пример токенизации: "немецкая классическая философия": {'input_ids': [37355, 2135, 5543, 3384, 33312], 'attention_mask': [1, 1, 1, 1, 1]}


Для автоматического паддинга и формирования батча будем использовать класс DataCollatorForLanguageModeling.

In [58]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [59]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Пример после токенизации

In [60]:
print(tokenizer.decode(train_dataset[3]))

 к исходному положению. Ведь если мы отнимем от сформированной вещи то, что сделало с ней орудие, то эта вёщь — в данном случае абсолютное — предстанет перед нами опять в том же самом виде, в каком она была и до этой, стало быть, ненужной, работы. Допустим, что орудие нужно вообще только для того, чтобы притянуть к себе с его помощью абсолютное, не внося в него при этом никаких изменений, — на манер того, как птичку притягивают палочкой, обмазанной клеем. В таком случае, если бы абсолютное само по себе еще не попало к нам


# Дообучение модели



In [61]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2").cuda()

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [62]:
training_args = TrainingArguments(
    output_dir = 'out',
    overwrite_output_dir = True,
    per_device_train_batch_size = 25,
    per_device_eval_batch_size = 25,
    learning_rate = 5e-5,
    num_train_epochs = 3,
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [63]:
trainer.train()

***** Running training *****
  Num examples = 4714
  Num Epochs = 3
  Instantaneous batch size per device = 25
  Total train batch size (w. parallel, distributed & accumulation) = 25
  Gradient Accumulation steps = 1
  Total optimization steps = 567


Step,Training Loss
500,3.458900


Saving model checkpoint to out/checkpoint-500
Configuration saved in out/checkpoint-500/config.json
Model weights saved in out/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=567, training_loss=3.4363591431309937, metrics={'train_runtime': 932.8247, 'train_samples_per_second': 15.16, 'train_steps_per_second': 0.608, 'total_flos': 923797979136000.0, 'train_loss': 3.4363591431309937, 'epoch': 3.0})

In [64]:
trainer.save_model()

Saving model checkpoint to out
Configuration saved in out/config.json
Model weights saved in out/pytorch_model.bin


## Генерация текста


In [65]:
generator = pipeline('text-generation', tokenizer="sberbank-ai/rugpt3small_based_on_gpt2", model='out')

loading configuration file out/config.json
Model config GPT2Config {
  "_name_or_path": "out",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file out/config.json
Model config GPT2Config {
  "_name_or_path": "out",
  "activ

Пара тестовых примеров, чтобы проверить адекватность модели:

In [86]:
print(generator('Классические проблемы философии:',
                                 max_length=50,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Классические проблемы философии: 
Что такое идеализм?
Сущность или сущность как предмет мышления.
Классическая философия, или философия разума, как мы видели, должна была бы называться философией разума. Только философия разума дает всю


In [74]:
print(generator('Бытие и ничто - две категории',
                                 max_length=80,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.8)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Бытие и ничто - две категории, в которых трансцендентальная диалектика может быть представлена как односторонняя; таким образом, она есть способность воспринять в виде понятия то, что лишь может быть понято. Но это единство, которое только что было показано при объяснении единства, оказывается недоступным трансцендентальному пониманию, так как в этой возможности содержится снятость  априорного единства


Seems fine

# Немного веселья :)

## Лучшие промты (с разных попыток):

* Как говаривал мой дед: "Все, что есть, должно быть, и все, что есть еще, должно быть еще"

* Как говаривал мой дед: "Счастье есть только в смерти".

* Как говаривал мой дед: "Ничто так не убивает, как смерть"

* Двач - это негатив, или чистое здравомыслие, которое не может быть иначе его. 

* Двачевание есть наивысшая форма бытия.

* Чрезмерный просмотр аниме влечет за собой различные отклонения вкуса;

* Заходят как-то аморал, нигилист и циник в бар. А бармен им: "Заратустра, плыви отсюда!"

* Главная проблема зумеров - это то, что они не могут сказать ничего вразумительного относительно явлений, поскольку они не могут познать причину их, так как они не знают, каковы причины явлений. 



In [105]:
res = generator('Как говаривал мой дед:',
                        max_length=25,
                        top_k=0,
                        temperature=0.6,
                        do_sample=True,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Как говаривал мой дед: "Всю жизнь должен я пройти, как тень, чтобы не стать тенью!""Всю 

Как говаривал мой дед: "Какое счастье родиться среди людей!"
Клянусь честью, я не могу сказать вам 

Как говаривал мой дед: "Если я хочу жить, я должен умереть".И я ответил: "Но я хочу умереть 

Как говаривал мой дед: "Смотри, я нашел свое счастье в том, что я люблю".Но я не нашел 

Как говаривал мой дед: "Ничто так не убивает, как смерть".
Но так как я люблю жизнь, 



In [199]:
res = generator('Двач - это',
                        max_length=30,
                        top_k=0,
                        do_sample=True,
                        temperature=0.8,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Двач - это действие; оно, конечно, снизу вверх, имеет в последнем значении рефлексию, но не внутри себя.
"Чистый 

Двач - это бытие, которое в себе определен как некоторое пустое значение; оно пустое значение, но определен как определенное, ибо ему присуща определенность 

Двач - это то же самое, что и отрицательное; положительное есть в себе, а отрицательное - в себе. И наоборот, отрицательное 

Двач - это лишь одно из них, затемнение же - его отсутствие. Первоначальные, в которых они развертываются, составляют лишь форму некоторой 

Двач - это просто другое содержание; в нем нет определенности. Но это отрицание уже не только снято, но и его истина есть его 



In [172]:
res = generator('Двачевание есть наивысшая форма',
                        max_length=30,
                        top_k=0,
                        do_sample=True,
                        temperature=0.8,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Двачевание есть наивысшая форма, из которой состоит восприятие, то есть ее суть - это форма представления. Следовательно, представление есть не что иное, 

Двачевание есть наивысшая форма разума, и его бесконечное развитие приводит к всеобщему единству постигающего. Как раз в этом единстве 

Двачевание есть наивысшая форма противоречия. Итак, мы имеем дело с открытым зрителем и раскрываемым всем человеческим существом; это бесстыдство самос 

Двачевание есть наивысшая форма бытия; проповедуя ее, Оно оправдывает себя тем, что оно предоставляет себе в качестве истины самости 

Двачевание есть наивысшая форма свободы ; для свободного есть нечто низшее, чем то, что для него и для него есть, и то, 



In [90]:
res = generator('Главная проблема зумеров',
                        max_length=70,
                        top_k=0,
                        do_sample=True,
                        temperature=0.8,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Главная проблема зумеровских исповеданий состоит в том, чтобы иметь представление, вне всякого понятия  о таких вещах, и притом о вещах, которые были бы возможны только в явлении, то есть о вещах, которые сами по себе не существуют. Какими бы бесконечными ни были наши представления, воображение, которая в явлении представляет себе 

Главная проблема зумеров - это то, что они не могут сказать ничего вразумительного относительно явлений, поскольку они не могут познать причину их, так как они не знают, каковы причины явлений. Так как это не касается их самих, они не могут также сказать, почему они не должны быть определены в опыте этого мира, и в этом вопросе им 

Главная проблема зумеровской физики состоит в том, что вся активность, все выходящие из нее явления, все, что направлено к цели, к познанию, к знанию возможно лишь благодаря тому, что эти явления суть лишь явления.
Но для этого необходимо нечто большее, чем эта необходимость. Если мы хотим увидеть, как в природе происходит 

Гла

In [92]:
res = generator('Чрезмереный просмотр аниме влечет',
                        max_length=45,
                        top_k=0,
                        temperature=0.8,
                        do_sample=True,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Чрезмереный просмотр аниме влечет за собой различные последствия для личности и, наоборот, для физического и душевного существования. Но в том и заключается неустанное требование прогресса и прогресса разума, что в основе всякого развития лежит и 

Чрезмереный просмотр аниме влечет за собой тяжелые последствия.
о, где же можно было оставить в таком случае такое мудрое замечание?
Мудрость та, что не имеет предиката, ибо она не имеет 

Чрезмереный просмотр аниме влечет за собой определенную степень влияния на людей, и отсюда вытекает, что каждое из них, равно как и любое другое, становится предметом всеобщих обсуждений.
Оно, следовательно, должно быть 

Чрезмереный просмотр аниме влечет за собой неистинное знание о предмете и природе этого явления, которой созерцается сливающееся с ним отношение, иначе говоря, она не есть знание о предмете.
Об 

Чрезмереный просмотр аниме влечет за собой, с одной стороны, порождение более высоких чувств, а с другой стороны, основной прием, которым пользу

In [160]:
res = generator('Заходят как-то аморал, нигилист и циник в бар. А бармен им:',
                        max_length=35,
                        top_k=0,
                        temperature=0.9,
                        do_sample=True,
                        num_return_sequences=5)
for item in res:
  print(item['generated_text'], '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Заходят как-то аморал, нигилист и циник в бар. А бармен им: "Заратустра, плыви отсюда!" И они 

Заходят как-то аморал, нигилист и циник в бар. А бармен им: "О, прекрасные братья мои, как же много у нас 

Заходят как-то аморал, нигилист и циник в бар. А бармен им: я доставляю удовольствие другим!
Но положительно никто из них 

Заходят как-то аморал, нигилист и циник в бар. А бармен им: "У нас неприлично ретироваться!"
О! 

Заходят как-то аморал, нигилист и циник в бар. А бармен им: Не о пожирающих, как лев, ходят речи; желуд 

